In [163]:
from serial import Serial
from pyubx2 import UBXReader, NMEA_PROTOCOL, UBX_PROTOCOL
from pyubx2 import UBXMessage, SET_LAYER_RAM, TXN_NONE, POLL
from pyubx2 import POLL_LAYER_RAM

In [35]:
usb_port = '/dev/ttyACM0'
baud_rate = 38400
set_layers = SET_LAYER_RAM
poll_layers = POLL_LAYER_RAM
transaction = TXN_NONE

In [183]:
def msg_send(msg):
    with Serial(usb_port, baud_rate, timeout=3) as stream:
        stream.write(msg)
        
def ubx_set(cfgData, layers=set_layers, transaction=transaction, verbose=False):
    msg = UBXMessage.config_set(layers, transaction, cfgData)
    if verbose:
        print(msg)
    else:
        msg_send(msg.serialize())

def ubx_poll(keys, layers=poll_layers,position=0, verbose=False):
    msg = UBXMessage.config_poll(layers, position, keys)
    if verbose:
        print(msg)
    else:
        with Serial(usb_port, baud_rate, timeout=3) as stream:
            stream.write(msg.serialize())
            ubr = UBXReader(stream, protfilter=NMEA_PROTOCOL | UBX_PROTOCOL)
            raw_data, parsed_data = ubr.read()
            return parsed_data

def ubx_reader(id=None, usb_port=usb_port, baud_rate=baud_rate, verbose=False, timeout=10):
    with Serial(usb_port, baud_rate, timeout=3) as stream:
        t = 0
        while True:
            t = t + 1
            if t > timeout:
                return None
            ubr = UBXReader(stream, protfilter=NMEA_PROTOCOL | UBX_PROTOCOL)
            raw_data, parsed_data = ubr.read()
            if parsed_data is not None:
                if (id is None or parsed_data.identity == id) and verbose:
                    print(parsed_data)
                if id is None or parsed_data.identity == id:
                    return parsed_data
                else:
                    continue


In [189]:
FixTypeList = ['no-fix', 'dead-reckoning-only', '2D-fix', '3D-fix', 'GPS + dead-reckoning', 'Time-only-fix']
SvinVldStatusList = ['not valid', 'valid']
SvinActiveStatusList = ['stop', 'in-progress']

def poll_nav_clk():
    msg = UBXMessage(b'\x01', b'\x22', POLL)
    msg_send(msg.serialize())
    msg = ubx_reader(id='NAV-CLOCK', verbose=True)
    return msg
    
def get_svin_status():
    msg = ubx_reader(id='TIM-SVIN')
    return SvinVldStatusList[msg.valid], SvinActiveStatusList[msg.active]

def get_fixtype(): 
    msg = ubx_reader(id='NAV-PVT')
    return FixTypeList[msg.fixType]

def set_coordinate(lon, lat, alt):
    cfgData = [('CFG_TMODE_LON', int(lon*10**7)), ('CFG_TMODE_LON_HP', int(lon*10**9)), 
            ('CFG_TMODE_LAT', int(lat*10**7)), ('CFG_TMODE_LAT_HP', int(lat*10**9)),
            ('CFG-TMODE-HEIGHT', int(alt*100)), ('CFG-TMODE-HEIGHT_HP', int(alt*10**4))]
    ubx_set(cfgData)
    
def set_acc(acc):
    cfgData = ['CFG-TMODE-FIXED_POS_ACC', acc*10**4]
    ubx_set(cfgData)

def set_rtcm():
    cfgData = [('CFG_MSGOUT_RTCM_3X_TYPE1005_USB', 1),
               ('CFG_MSGOUT_RTCM_3X_TYPE1077_USB', 1),
               ('CFG_MSGOUT_RTCM_3X_TYPE1087_USB', 1),
               ('CFG_MSGOUT_RTCM_3X_TYPE1097_USB', 1),
               ('CFG_MSGOUT_RTCM_3X_TYPE1127_USB', 1),
               ('CFG_MSGOUT_RTCM_3X_TYPE1230_USB', 1),
               ('CFG_MSGOUT_RTCM_3X_TYPE4072_1_USB', 1)]
    ubx_set(cfgData)

def get_coordindate():
    msg = ubx_reader(id='NAV-PVT')
    lon = msg.lon
    lat = msg.lat
    alt = msg.height
    return lon, lat, alt

def get_acc():
    msg = ubx_reader(id='NAV-PVT')
    hacc = msg.hAcc/1000
    vacc = msg.vAcc/1000
    return hacc, vacc

def master_io_msg():
    cfgData = [('CFG_USBINPROT_UBX',  1),
               ('CFG_USBINPROT_NMEA', 1),
               ('CFG_USBOUTPROT_UBX', 0),
               ('CFG_USBOUTPROT_NMEA',0),
               ('CFG_USBOUTPROT_RTCM',1)]
    ubx_set(cfgData)

def slave_io_msg():
    cfgData = [('CFG_USBINPROT_UBX',  1),
               ('CFG_USBINPROT_NMEA', 1),
               ('CFG_USBINPROT_RTCM', 1),
               ('CFG_USBOUTPROT_UBX', 1),
               ('CFG_USBOUTPROT_NMEA',1)]
    ubx_set(cfgData)

In [78]:
# USB input: NMEA, UBX 
keys = [("CFG_USBINPROT_NMEA", 1), ("CFG_USBINPROT_UBX", 1)]
ubx_set(keys, verbose=True)
# USB output: UBX
keys = [("CFG_USBOUTPROT_NMEA", 0), ("CFG_USBOUTPROT_UBX", 1)]
ubx_set(keys, verbose=False)



<UBX(CFG-VALSET, version=0, ram=1, bbr=0, flash=0, action=0, reserved0=0, CFG_USBINPROT_NMEA=1, CFG_USBINPROT_UBX=1)>


In [44]:
keys=["CFG_USBOUTPROT_NMEA"]
ubx_poll(keys, verbose=True)

<UBX(CFG-VALGET, version=0, layer=0, position=0, keys_01=276299778)>


In [47]:
keys=["CFG_USBOUTPROT_NMEA"]
ubx_poll(keys)

<UBX(CFG-VALGET, version=1, layer=0, position=0, CFG_USBOUTPROT_NMEA=0)>


In [70]:
# set the GPS receiver work in survey-in mode
# survey-in minimum duration is 1200s
# survey-in position accuracy limit 0.1m
keys = [("CFG_TMODE_MODE", 1), ("CFG_TMODE_SVIN_MIN_DUR", 1200), ("CFG_TMODE_SVIN_ACC_LIMIT", 1000)]
ubx_set(keys, verbose=False)

In [92]:
# enable ubx-pvt message out from usb
keys = [("CFG_MSGOUT_UBX_NAV_PVT_USB",1), ("CFG_MSGOUT_UBX_TIM_SVIN_USB", 1)]
ubx_set(keys)

In [88]:
msg = ubx_reader(verbose=True)
#print(msg.gnssFixOk)

<UBX(NAV-PVT, iTOW=02:53:47, year=2024, month=6, day=26, hour=2, min=53, second=47, validDate=1, validTime=1, fullyResolved=1, validMag=0, tAcc=42, nano=-159915, fixType=3, gnssFixOk=1, diffSoln=0, psmState=0, headVehValid=0, carrSoln=0, confirmedAvai=1, confirmedDate=1, confirmedTime=1, numSV=12, lon=-122.257403, lat=37.8727853, height=88528, hMSL=117962, hAcc=7321, vAcc=8348, velN=4, velE=-19, velD=-41, gSpeed=19, headMot=253.38518, sAcc=36, headAcc=30.44452, pDOP=3.32, invalidLlh=0, lastCorrectionAge=0, reserved0=659535132, headVeh=0.0, magDec=0.0, magAcc=0.0)>


In [86]:
msg = ubx_reader(verbose=True)

<UBX(TIM-SVIN, dur=5320, meanX=-269058963, meanY=-426311744, meanZ=389436953, meanV=1554856064, obs=5321, valid=0, active=1, reserved1=0)>


In [112]:
ubx_reader(id='TIM-SVIN' ,verbose=True)

<UBX(TIM-SVIN, dur=6303, meanX=-269058991, meanY=-426311767, meanZ=389436894, meanV=1556117376, obs=6304, valid=0, active=1, reserved1=0)>


UBXMessage(b'\r', b'\x04', 0, payload=b'\x9f\x18\x00\x00Q|\xf6\xef\xa9\xff\x96\xe6\xdeU6\x17\x80w\xc0\\\xa0\x18\x00\x00\x00\x01\x00\x00')

In [113]:
# enable ubx messages
keys = [("CFG_MSGOUT_UBX_NAV_PVT_USB", 1), 
        ("CFG_MSGOUT_UBX_NAV_SAT_USB", 1), 
        ("CFG_MSGOUT_UBX_NAV_SIG_USB", 1), 
        ("CFG_MSGOUT_UBX_NAV_STATUS_USB", 1),
        ("CFG_MSGOUT_UBX_RXM_RTCM_USB", 1)]
ubx_set(keys)

In [123]:
ubx_reader(id='NAV-PVT' ,verbose=True)

<UBX(NAV-PVT, iTOW=03:58:51, year=2024, month=6, day=26, hour=3, min=58, second=51, validDate=1, validTime=1, fullyResolved=1, validMag=0, tAcc=46, nano=-482769, fixType=3, gnssFixOk=1, diffSoln=0, psmState=0, headVehValid=0, carrSoln=0, confirmedAvai=1, confirmedDate=1, confirmedTime=1, numSV=13, lon=-122.2570436, lat=37.8729617, height=83080, hMSL=112513, hAcc=7937, vAcc=8155, velN=1, velE=24, velD=-22, gSpeed=24, headMot=253.38518, sAcc=10, headAcc=30.44452, pDOP=1.93, invalidLlh=0, lastCorrectionAge=0, reserved0=659535132, headVeh=0.0, magDec=0.0, magAcc=0.0)>


UBXMessage(b'\x01', b'\x07', 0, payload=b"\xc8\x06N\x10\xe8\x07\x06\x1a\x03:37.\x00\x00\x00/\xa2\xf8\xff\x03\x01\xea\r<\x0e!\xb7\x91\xf4\x92\x16\x88D\x01\x00\x81\xb7\x01\x00\x01\x1f\x00\x00\xdb\x1f\x00\x00\x01\x00\x00\x00\x18\x00\x00\x00\xea\xff\xff\xff\x18\x00\x00\x00\x96\xa2\x82\x01\n\x00\x00\x00dt.\x00\xc1\x00\x00\x00\x1c\xb5O'\x00\x00\x00\x00\x00\x00\x00\x00")

In [79]:
get_fixtype()

'3D-fix'

In [80]:
get_svin_status()

('not valid', 'in-progress')

In [81]:
get_coordindate()

(-122.2570535, 37.8728349, 93.322)

In [77]:
get_acc()

(0.486, 0.819)

In [190]:
poll_nav_clk()

<UBX(NAV-CLOCK, iTOW=04:40:28, clkB=689796, clkD=83, tAcc=23, fAcc=701)>


UBXMessage(b'\x01', b'"', 0, payload=b'\xb0 t\x10\x84\x86\n\x00S\x00\x00\x00\x17\x00\x00\x00\xbd\x02\x00\x00')